## Read in planned figures and then join expected, planned and actual figures to final data set

In [1]:
import os, warnings
#warnings.filterwarnings("ignore")
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import pandas as pd
import numpy as np
from copy import deepcopy
import datetime
pd.set_option('display.max_rows', 10**5)
pd.set_option('display.max_columns', 10**5)

from pdb import set_trace
from importlib import reload
import adtk
from tqdm.auto import tqdm
#import pweave     # for markdown reports

os.chdir("..")
from claims_reporting.utils import tsa_utils as tsa
from claims_reporting.utils import utils_func as util
from claims_reporting.services import file

from claims_reporting.config import global_config as glob
from claims_reporting.services import file
#from claims_reporting.services import base
from claims_reporting.resources import config

reload(tsa)
reload(file)
reload(config)
#reload(base)
reload(glob)

#os.getcwd()
#reload(util)


<module 'claims_reporting.config.global_config' from '/work/claims-anomaly-financial/src/claims_reporting/config/global_config.py'>

Just some trials...

In [ ]:
os.listdir()

year = "2021"

fpath = glob.UC_DATA_DIR + "Planned amounts - GPS" + '/'+str(year)
filename = os.listdir(fpath)[0]   # assuming we have only one file in the folder
filename

In [ ]:
xls = file.XLSXService(path=filename, root_path=fpath, sheetname='GrossNet', header=[2, 3,4, 5], index_col=None)
data_orig = xls.doRead()
#data_orig.drop(columns=data_orig.columns[0], inplace=True)
#data_orig = data_orig.T     # transpose data!
print(data_orig.shape)

In [ ]:
xls = file.XLSXService(path=filename, root_path=fpath, sheetname='GrossNet',header=[1,2,3,4, 5], index_col=None)
data_orig = xls.doRead()
data_orig.drop(columns=data_orig.columns[0], inplace=True)
data_orig = data_orig.T     # transpose data!
print(data_orig.shape)

In [ ]:
#data_orig = data_orig.T     # transpose data!

In [ ]:
data_orig.head(100)

In [ ]:
LoB = data_orig.iloc[0,:].values
OE = data_orig.iloc[1,:].values
Region = data_orig.iloc[2,:].values
df = data_orig[3:] 

#df[1]

In [ ]:
df.head(100)

In [ ]:
len(df.index)

In [ ]:
df.index[0]

In [ ]:
df.shape

In [ ]:
i = 0
row_index = df.index[i]
row_index

In [ ]:
main, layer, toi, s, _ = row_index

In [ ]:

for i in range(len(df.index)):
    row_index = df.index[i]
    main, layer, toi, s, _ = row_index
    #print(main, s)

In [ ]:
S = 12
file_input_years = 2021

LoB = data_orig.iloc[0,:].values
OE = data_orig.iloc[1,:].values
Region = data_orig.iloc[2,:].values
df = data_orig[3:] 

full_flat = pd.DataFrame(columns=['LoB', 'OE', 'Region', 'Layer', 'Type of Information','12M', 'Year','Month', 'Time','EUR'])
for column in tqdm(range(1,df.shape[1])):

    flat_data = pd.DataFrame(columns=['LoB', 'OE', 'Region', 'Layer', 'Type of Information', '12M','Year', 'Month', 'Time', 'EUR'])
    periods = [str(s) for s in range(0,S+1)]
    values = df[column].values    # column 1
    z=0
    for i in range(len(df.index)):
        row_index = df.index[i]
        main, layer, toi, s, _ = row_index
        
        if s in periods:
            time = "{:4d}-{:02d}".format(int(file_input_years),int(s))
            flat_data.loc[z,:] = [LoB[column], OE[column], Region[column], layer, toi, main, file_input_years, s, time, values[i]]
            z += 1
    full_flat = pd.concat([full_flat, flat_data], axis=0)        

print(full_flat.shape)

In [ ]:
full_flat.tail(10)

In [ ]:
#data_orig.columns = data_orig.columns.to_flat_index()

## Loop over all xls-files:

In [2]:
full_flat = pd.DataFrame(columns=['LoB', 'OE', 'Region', 'Layer', 'Type of Information', '12M','Year','Month', 'Time','EUR'])
S = 12

print("Overall progress:")
for file_input_years in tqdm(['2018', '2019', '2020', '2021']):
    
    print('Year: {}'.format(file_input_years))
    fpath = glob.UC_DATA_DIR + "Planned amounts - GPS" + '/'+str(file_input_years)
    filename = os.listdir(fpath)[0]   # assuming we have only one file in the folder
    print("Importing {}".format(filename))
    
    # Import file
    xls = file.XLSXService(path=filename, root_path=fpath, sheetname='GrossNet', header=[1, 2, 3,4, 5], index_col=None)
    data_orig = xls.doRead()
    data_orig.drop(columns=data_orig.columns[0], inplace=True)
    data_orig = data_orig.T     # transpose data!
    print('Size {}'.format(data_orig.shape))
    
    LoB = data_orig.iloc[0,:].values
    OE = data_orig.iloc[1,:].values
    Region = data_orig.iloc[2,:].values
    df = data_orig[3:] 
    
    # Loop over all combinations of 'LoB', 'OE', 'Region' ('columns')
    for column in tqdm(range(0,df.shape[1])):

        flat_data = pd.DataFrame(columns=['LoB', 'OE', 'Region', 'Layer', 'Type of Information', '12M', 'Year', 'Month', 'Time', 'EUR'])
        periods = [str(s) for s in range(1,S+1)]
        values = df[column].values    # column 1
        z=0
        for i in range(len(df.index)):
            row_index = df.index[i]
            main, layer, toi, s, _ = row_index

            if s in periods:
                time = "{:4d}-{:02d}".format(int(file_input_years),int(s))
                flat_data.loc[z,:] = [LoB[column], OE[column], Region[column], layer, toi,main, file_input_years, s, time, values[i]]
                z += 1
        full_flat = pd.concat([full_flat, flat_data], axis=0)        

    print(full_flat.shape)
    

Overall progress:


Year: 2018
Importing 2018 Net GrossNet.xlsx
Size (83, 2472)



(177984, 10)
Year: 2019
Importing Net and GrossNet Claims 2019A and 2019P by month.xlsx
Size (83, 2697)



(372168, 10)
Year: 2020
Importing Net and GrossNet Claims 2020A and 2020P by month.xlsx
Size (83, 2180)



(529128, 10)
Year: 2021
Importing Net and GrossNet Claims 2021A and 2021P by month.xlsx
Size (83, 2845)



(733968, 10)



In [3]:
print(full_flat.shape)

(733968, 10)


In [62]:
# Create filters:
planned_filter = full_flat['12M'].str.contains(pat = 'P 12M')   # only use planned time series
four_region_filter = full_flat['Region'].str.len() == 4

In [63]:
final = full_flat[four_region_filter & planned_filter]
planned = final.rename(columns={'LoB' : 'Sub Line of Business'})

Load readily prepared planned data:

In [104]:
# Load readily prepared planned data:
#-------------------------------------
pkl = file.PickleService(path = "planned.pkl", root_path = glob.UC_DATA_DIR)
#pkl.doWrite(planned)
planned = pkl.doRead()
planned.shape

Pickle Service Read from File: /data/claims_reporting/input_data/planned.pkl


(226800, 10)

Only keep Non-NaN rows:

In [105]:
na_mask = planned[['Time', 'EUR']].notnull().all(axis=1)
planned = planned[na_mask]
print(planned.shape)

(138880, 10)


Replace missings by zero to keep more rows during merge (as we later cut out NANs from left join):

In [106]:
#planned['EUR'] = planned['EUR'].fillna(0)

In [107]:
#pg = file.PostgresService(verbose=False)

#pg.doWrite(planned, "Planned_CAY_PAY")    # if_exists = "append"

#planned = pg.doRead(qry = 'select * from "Planned_CAY_PAY"')   
#planned.shape

Read incurred + actuarial PAY/CAY:

In [108]:
pkl = file.PickleService(path = "Incurred_Expected_CAY_PAY.pkl", root_path = glob.UC_DATA_DIR)
#pkl.doWrite(dm_new)
inc_exp = pkl.doRead()
print(inc_exp.shape)

Pickle Service Read from File: /data/claims_reporting/input_data/Incurred_Expected_CAY_PAY.pkl
(114602, 42)


In [109]:
from claims_reporting.resources import config

reload(config)

lookup = config.mappings['planned_to_cll']
lookup2 = config.mappings['cll']
lookup2

{'Branch Code': {'London': 'UK01', 'Milton Keynes': 'UK01'},
 'OE': {'INDIA': 'India'}}

In [110]:
for k in lookup.keys():
    print(k)
    planned[k] = planned[k].replace(lookup[k])

Sub Line of Business
OE


In [111]:
for k in lookup2.keys():
    print(k)
    inc_exp[k] = inc_exp[k].replace(lookup2[k])

Branch Code
OE


In [112]:
planned['Region'] = planned['Region'].str[:2]
inc_exp['Actuarial Branch'] = inc_exp['Actuarial Branch'].str[:2]
inc_exp['Branch Code'] = inc_exp['Branch Code'].str[:2]
print(planned.shape)
planned = planned[~planned['Sub Line of Business'].str.contains('TOTAL')]   # drop TOTAL* LoB (=aggregated)
#planned = planned[planned['Type of Information'] == 'CLAIMS INCURRED C.A.']
planned.columns = plan_cay_names = [s+' - Plan' for s in planned.columns]
print(planned.shape)

(138880, 10)
(82686, 10)


In [113]:
#planned[planned['Type of Information - Plan'] == 'CLAIMS INCURRED C.A.'].head(20)

In [114]:
planned_cay = planned[planned['Type of Information - Plan'] == 'CLAIMS INCURRED C.A.']
planned_cay.columns = plan_cay_names = [s+' - '+'CAY' for s in planned.columns]
print(planned.shape)
print(planned_cay.shape)

(82686, 10)
(28343, 10)


In [115]:
planned_pay = planned[planned['Type of Information - Plan'] == 'CLAIMS INCURRED P.A.']
planned_pay.columns = plan_pay_names = [s+' - '+'PAY' for s in planned.columns]
print(planned.shape)
print(planned_pay.shape)

(82686, 10)
(24706, 10)


Join Incurred-Expected data with Planned CAY:

In [120]:
final_data = inc_exp.merge(planned_cay, how = "left", 
                           left_on = ['Actuarial Sub Line of Business', 
                                                      'Branch Code', 'time - CAY',
                                                      'OE'
                                                     ],
             right_on = ['Sub Line of Business - Plan - CAY', 
                         'Region - Plan - CAY', 'Time - Plan - CAY',
                         'OE - Plan - CAY'
                        ])

na_mask = final_data.notnull().all(axis=1)
#final_data_new = final_data[na_mask]
final_data_new = deepcopy(final_data)


print(inc_exp.shape)
print(final_data.shape)
print(final_data_new.shape)

(114602, 42)
(129482, 52)
(129482, 52)


In [121]:
final_data.head(30)

,Region,OE,Line of Business,Sub Line of Business,Source System,Actuarial Region,Actuarial Sub Line of Business,Actuarial Branch,Branch Code,Legal Entity,time,Loss Category,Accident year,target,Legal Entity - CAY,Branch - CAY,Region - CAY,Line of Business - CAY,Sub Line of Business - CAY,time - CAY,Legal Entity - PAY,Branch - PAY,Region - PAY,Line of Business - PAY,Sub Line of Business - PAY,time - PAY,ExpMovAttr_PAY,ExpMovLg_PAY,ExpMovMed_PAY,ExpMovCat_PAY,ExpMovAttr_CAY,ExpMovLg_CAY,ExpMovMed_CAY,ExpMovCat_CAY,ExpMovAttr_PAY_diffs,ExpMovLg_PAY_diffs,ExpMovMed_PAY_diffs,ExpMovCat_PAY_diffs,ExpMovAttr_CAY_diffs,ExpMovLg_CAY_diffs,ExpMovMed_CAY_diffs,ExpMovCat_CAY_diffs,Sub Line of Business - Plan - CAY,OE - Plan - CAY,Region - Plan - CAY,Layer - Plan - CAY,Type of Information - Plan - CAY,12M - Plan - CAY,Year - Plan - CAY,Month - Plan - CAY,Time - Plan - CAY,EUR - Plan - CAY
0,NaN,ART Swiss,ART,ART LOB,RoW,CEE_CH_LI,ART LOB,CH,NaN,AGCS ART,2018-12,3-Large,2018,5.824451e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Asia Pacific,ARAP,Engineering,Engineering Main,RoW,AP_BRANCHES,EC Construction,EX,NaN,AGCS SE,2019-10,2-Medium,2010,7.256222e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Asia Pacific,ARAP,Engineering,Engineering Main,RoW,NaN,EC Construction,NaN,NaN,NaN,2019-10,2-Medium,2011,1.707132e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Asia Pacific,Australia,Aviation,Aviation Airlines,Regent-Australia,AP_PACIFIC,Aviation General,DE,AU,AGCS SE,2018-01,1-Attritional,2014,-2.010192e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Asia Pacific,Australia,Aviation,Aviation Airlines,Regent-Australia,AP_PACIFIC,Aviation General,DE,AU,AGCS SE,2018-01,4-Cat,2017,8.229671e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Asia Pacific,Australia,Aviation,Aviation Airlines,Regent-Australia,AP_PACIFIC,Aviation General,DE,AU,AGCS SE,2018-02,1-Attritional,2013,1.020637e+05,AGCS SE,DE,AP_PACIFIC,Aviation,Aviation General,2018-02,AGCS SE,DE,AP_PACIFIC,Aviation,Aviation General,2018-02,8.448000e+01,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000,0.0,0.000000,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000,0.000000,0.0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,Financial Lines,Professional Indemnity,Cognos,NA_US,FL Professional Indemnity,US,US,AGCS NA,2020-01,1-Attritional,2019,3.372966e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,ART Swiss,ART,ART LOB,RoW,CEE_CH_LI,ART LOB,CH,NaN,AGCS ART,2018-06,4-Cat,2017,-2.023779e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,ART,ART LOB,Cognos,NA_US,ART LOB,US,US,AGCS NA,2020-03,1-Attritional,2020,4.216207e+00,AGCS NA,US,NA_US,ART,ART LOB,2020-03,AGCS NA,US,NA_US,ART,ART LOB,2020-03,7.145293e+07,8.356961e+07,0.0,8.412683e+07,5.559606e+01,0.000000,0.0,0.000000,6.848974e+07,8.150755e+07,0.0,8.391348e+07,41.029420,0.000000,0.0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,ART Swiss,ART,ART LOB,RoW,CEE_CH_LI,ART LOB,CH,NaN,AGCS ART,2018-09,4-Cat,2017,1.152368e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Join Incurred-Expected data with Planned PAY:

In [126]:

final_data_set = final_data.merge(planned_pay, how = "inner", 
                           left_on = ['Actuarial Sub Line of Business', 
                                      'Branch Code', 'time - PAY',
                                      'OE'],
             right_on = ['Sub Line of Business - Plan - PAY', 
                         'Region - Plan - PAY', 'Time - Plan - PAY',
                         'OE - Plan - PAY'
                        ])

na_mask = final_data_set[['EUR - Plan - PAY', 'EUR - Plan - CAY']].notnull().all(axis=1)
#final_data_set2 = final_data_set[na_mask]
final_data_set2 = deepcopy(final_data_set)

final_data_set = final_data_set.rename(columns={'target' : 'Incurred amount EUR'})
final_data_set2 = final_data_set2.rename(columns={'target' : 'Incurred amount EUR'})

print(final_data.shape)
#print(final_data_new.shape)
print(final_data_set.shape)
print(final_data_set2.shape)


(129482, 52)
(90419, 62)
(90419, 62)


### Save final data set:

In [127]:
pkl = file.PickleService(path = "Incurred_Expected_Planned_FULL.pkl", root_path = glob.UC_DATA_DIR)
pkl.doWrite(final_data_set2)
#final_data_set = pkl.doRead()
print(final_data_set2.shape)

Pickle Service Output to File: /data/claims_reporting/input_data/Incurred_Expected_Planned_FULL.pkl
(90419, 62)


In [128]:
pg = file.PostgresService(verbose=False)

pg.doWrite(final_data_set2, "Incurred_Expected_Planned_FULL")    

In [99]:
final_data_set.head(30)

,Region,OE,Line of Business,Sub Line of Business,Source System,Actuarial Region,Actuarial Sub Line of Business,Actuarial Branch,Branch Code,Legal Entity,time,Loss Category,Accident year,Incurred amount EUR,Legal Entity - CAY,Branch - CAY,Region - CAY,Line of Business - CAY,Sub Line of Business - CAY,time - CAY,Legal Entity - PAY,Branch - PAY,Region - PAY,Line of Business - PAY,Sub Line of Business - PAY,time - PAY,ExpMovAttr_PAY,ExpMovLg_PAY,ExpMovMed_PAY,ExpMovCat_PAY,ExpMovAttr_CAY,ExpMovLg_CAY,ExpMovMed_CAY,ExpMovCat_CAY,ExpMovAttr_PAY_diffs,ExpMovLg_PAY_diffs,ExpMovMed_PAY_diffs,ExpMovCat_PAY_diffs,ExpMovAttr_CAY_diffs,ExpMovLg_CAY_diffs,ExpMovMed_CAY_diffs,ExpMovCat_CAY_diffs,Sub Line of Business - Plan - CAY,OE - Plan - CAY,Region - Plan - CAY,Layer - Plan - CAY,Type of Information - Plan - CAY,12M - Plan - CAY,Year - Plan - CAY,Month - Plan - CAY,Time - Plan - CAY,EUR - Plan - CAY,Sub Line of Business - Plan - PAY,OE - Plan - PAY,Region - Plan - PAY,Layer - Plan - PAY,Type of Information - Plan - PAY,12M - Plan - PAY,Year - Plan - PAY,Month - Plan - PAY,Time - Plan - PAY,EUR - Plan - PAY
0,North America,US,ART,ART LOB,Cognos,NA_US,ART LOB,US,US,AGCS NA,2020-03,1-Attritional,2019,-0.843241,AGCS NA,US,NA_US,ART,ART LOB,2020-03,AGCS NA,US,NA_US,ART,ART LOB,2020-03,7.145293e+07,8.356961e+07,0.0,8.412683e+07,5.559606e+01,0.000000,0.0,0.000000,6.848974e+07,8.150755e+07,0.0,8.391348e+07,41.029420,0.000000,0.0,0.000000,ART LOB,US,US,GROSS NET FULLY CONS,CLAIMS INCURRED C.A.,P 12M 2020,2020,3,2020-03,-1.64832e+06,ART LOB,US,US,GROSS NET FULLY CONS,CLAIMS INCURRED P.A.,P 12M 2020,2020,3,2020-03,0.02
1,North America,US,ART,ART LOB,Cognos,NA_US,ART LOB,US,US,AGCS NA,2020-03,1-Attritional,2019,-0.843241,AGCS NA,US,NA_US,ART,ART LOB,2020-03,AGCS NA,US,NA_US,ART,ART LOB,2020-03,7.145293e+07,8.356961e+07,0.0,8.412683e+07,5.559606e+01,0.000000,0.0,0.000000,6.848974e+07,8.150755e+07,0.0,8.391348e+07,41.029420,0.000000,0.0,0.000000,ART LOB,US,US,GROSS NET FULLY CONS,CLAIMS INCURRED C.A.,P 12M 2020,2020,3,2020-03,-1.64832e+06,ART LOB,US,US,GROSS NET FULLY CONS,CLAIMS INCURRED P.A.,P 12M 2020,2020,3,2020-03,-0.01
2,North America,US,ART,ART LOB,Cognos,NA_US,ART LOB,US,US,AGCS NA,2020-03,1-Attritional,2018,-4.216207,AGCS NA,US,NA_US,ART,ART LOB,2020-03,AGCS NA,US,NA_US,ART,ART LOB,2020-03,7.145293e+07,8.356961e+07,0.0,8.412683e+07,5.559606e+01,0.000000,0.0,0.000000,6.848974e+07,8.150755e+07,0.0,8.391348e+07,41.029420,0.000000,0.0,0.000000,ART LOB,US,US,GROSS NET FULLY CONS,CLAIMS INCURRED C.A.,P 12M 2020,2020,3,2020-03,-1.64832e+06,ART LOB,US,US,GROSS NET FULLY CONS,CLAIMS INCURRED P.A.,P 12M 2020,2020,3,2020-03,0.02
3,North America,US,ART,ART LOB,Cognos,NA_US,ART LOB,US,US,AGCS NA,2020-03,1-Attritional,2018,-4.216207,AGCS NA,US,NA_US,ART,ART LOB,2020-03,AGCS NA,US,NA_US,ART,ART LOB,2020-03,7.145293e+07,8.356961e+07,0.0,8.412683e+07,5.559606e+01,0.000000,0.0,0.000000,6.848974e+07,8.150755e+07,0.0,8.391348e+07,41.029420,0.000000,0.0,0.000000,ART LOB,US,US,GROSS NET FULLY CONS,CLAIMS INCURRED C.A.,P 12M 2020,2020,3,2020-03,-1.64832e+06,ART LOB,US,US,GROSS NET FULLY CONS,CLAIMS INCURRED P.A.,P 12M 2020,2020,3,2020-03,-0.01
4,North America,US,ART,ART LOB,FFIC,NA_US,ART LOB,US,US,AGCS NA,2020-03,1-Attritional,2020,4234.758411,AGCS NA,US,NA_US,ART,ART LOB,2020-03,AGCS NA,US,NA_US,ART,ART LOB,2020-03,7.145293e+07,8.356961e+07,0.0,8.412683e+07,5.559606e+01,0.000000,0.0,0.000000,6.848974e+07,8.150755e+07,0.0,8.391348e+07,41.029420,0.000000,0.0,0.000000,ART LOB,US,US,GROSS NET FULLY CONS,CLAIMS INCURRED C.A.,P 12M 2020,2020,3,2020-03,-1.64832e+06,ART LOB,US,US,GROSS NET FULLY CONS,CLAIMS INCURRED P.A.,P 12M 2020,2020,3,2020-03,0.02
5,North America,US,ART,ART LOB,FFIC,NA_US,ART LOB,US,US,AGCS NA,2020-03,1-Attritional,2020,4234.758411,AGCS NA,US,NA_US,ART,ART LOB,2020-03,AGCS NA,US,NA_US,ART,ART LOB,2020-03,7.145293e+07,8.356961e+07,0.0,8.412683e+07,5.559606e+01,0.000000,0.0,0.000000,6.848974e+07,8.150755e+07,0.0,8.391348e+07,41.029420,0.000000,0.0,0.000

In [100]:
final_data_set2.head(30)

,Region,OE,Line of Business,Sub Line of Business,Source System,Actuarial Region,Actuarial Sub Line of Business,Actuarial Branch,Branch Code,Legal Entity,time,Loss Category,Accident year,Incurred amount EUR,Legal Entity - CAY,Branch - CAY,Region - CAY,Line of Business - CAY,Sub Line of Business - CAY,time - CAY,Legal Entity - PAY,Branch - PAY,Region - PAY,Line of Business - PAY,Sub Line of Business - PAY,time - PAY,ExpMovAttr_PAY,ExpMovLg_PAY,ExpMovMed_PAY,ExpMovCat_PAY,ExpMovAttr_CAY,ExpMovLg_CAY,ExpMovMed_CAY,ExpMovCat_CAY,ExpMovAttr_PAY_diffs,ExpMovLg_PAY_diffs,ExpMovMed_PAY_diffs,ExpMovCat_PAY_diffs,ExpMovAttr_CAY_diffs,ExpMovLg_CAY_diffs,ExpMovMed_CAY_diffs,ExpMovCat_CAY_diffs,Sub Line of Business - Plan - CAY,OE - Plan - CAY,Region - Plan - CAY,Layer - Plan - CAY,Type of Information - Plan - CAY,12M - Plan - CAY,Year - Plan - CAY,Month - Plan - CAY,Time - Plan - CAY,EUR - Plan - CAY,Sub Line of Business - Plan - PAY,OE - Plan - PAY,Region - Plan - PAY,Layer - Plan - PAY,Type of Information - Plan - PAY,12M - Plan - PAY,Year - Plan - PAY,Month - Plan - PAY,Time - Plan - PAY,EUR - Plan - PAY
0,North America,US,ART,ART LOB,Cognos,NA_US,ART LOB,US,US,AGCS NA,2020-03,1-Attritional,2019,-0.843241,AGCS NA,US,NA_US,ART,ART LOB,2020-03,AGCS NA,US,NA_US,ART,ART LOB,2020-03,7.145293e+07,8.356961e+07,0.0,8.412683e+07,5.559606e+01,0.000000,0.0,0.000000,6.848974e+07,8.150755e+07,0.0,8.391348e+07,41.029420,0.000000,0.0,0.000000,ART LOB,US,US,GROSS NET FULLY CONS,CLAIMS INCURRED C.A.,P 12M 2020,2020,3,2020-03,-1.64832e+06,ART LOB,US,US,GROSS NET FULLY CONS,CLAIMS INCURRED P.A.,P 12M 2020,2020,3,2020-03,0.02
1,North America,US,ART,ART LOB,Cognos,NA_US,ART LOB,US,US,AGCS NA,2020-03,1-Attritional,2019,-0.843241,AGCS NA,US,NA_US,ART,ART LOB,2020-03,AGCS NA,US,NA_US,ART,ART LOB,2020-03,7.145293e+07,8.356961e+07,0.0,8.412683e+07,5.559606e+01,0.000000,0.0,0.000000,6.848974e+07,8.150755e+07,0.0,8.391348e+07,41.029420,0.000000,0.0,0.000000,ART LOB,US,US,GROSS NET FULLY CONS,CLAIMS INCURRED C.A.,P 12M 2020,2020,3,2020-03,-1.64832e+06,ART LOB,US,US,GROSS NET FULLY CONS,CLAIMS INCURRED P.A.,P 12M 2020,2020,3,2020-03,-0.01
2,North America,US,ART,ART LOB,Cognos,NA_US,ART LOB,US,US,AGCS NA,2020-03,1-Attritional,2018,-4.216207,AGCS NA,US,NA_US,ART,ART LOB,2020-03,AGCS NA,US,NA_US,ART,ART LOB,2020-03,7.145293e+07,8.356961e+07,0.0,8.412683e+07,5.559606e+01,0.000000,0.0,0.000000,6.848974e+07,8.150755e+07,0.0,8.391348e+07,41.029420,0.000000,0.0,0.000000,ART LOB,US,US,GROSS NET FULLY CONS,CLAIMS INCURRED C.A.,P 12M 2020,2020,3,2020-03,-1.64832e+06,ART LOB,US,US,GROSS NET FULLY CONS,CLAIMS INCURRED P.A.,P 12M 2020,2020,3,2020-03,0.02
3,North America,US,ART,ART LOB,Cognos,NA_US,ART LOB,US,US,AGCS NA,2020-03,1-Attritional,2018,-4.216207,AGCS NA,US,NA_US,ART,ART LOB,2020-03,AGCS NA,US,NA_US,ART,ART LOB,2020-03,7.145293e+07,8.356961e+07,0.0,8.412683e+07,5.559606e+01,0.000000,0.0,0.000000,6.848974e+07,8.150755e+07,0.0,8.391348e+07,41.029420,0.000000,0.0,0.000000,ART LOB,US,US,GROSS NET FULLY CONS,CLAIMS INCURRED C.A.,P 12M 2020,2020,3,2020-03,-1.64832e+06,ART LOB,US,US,GROSS NET FULLY CONS,CLAIMS INCURRED P.A.,P 12M 2020,2020,3,2020-03,-0.01
4,North America,US,ART,ART LOB,FFIC,NA_US,ART LOB,US,US,AGCS NA,2020-03,1-Attritional,2020,4234.758411,AGCS NA,US,NA_US,ART,ART LOB,2020-03,AGCS NA,US,NA_US,ART,ART LOB,2020-03,7.145293e+07,8.356961e+07,0.0,8.412683e+07,5.559606e+01,0.000000,0.0,0.000000,6.848974e+07,8.150755e+07,0.0,8.391348e+07,41.029420,0.000000,0.0,0.000000,ART LOB,US,US,GROSS NET FULLY CONS,CLAIMS INCURRED C.A.,P 12M 2020,2020,3,2020-03,-1.64832e+06,ART LOB,US,US,GROSS NET FULLY CONS,CLAIMS INCURRED P.A.,P 12M 2020,2020,3,2020-03,0.02
5,North America,US,ART,ART LOB,FFIC,NA_US,ART LOB,US,US,AGCS NA,2020-03,1-Attritional,2020,4234.758411,AGCS NA,US,NA_US,ART,ART LOB,2020-03,AGCS NA,US,NA_US,ART,ART LOB,2020-03,7.145293e+07,8.356961e+07,0.0,8.412683e+07,5.559606e+01,0.000000,0.0,0.000000,6.848974e+07,8.150755e+07,0.0,8.391348e+07,41.029420,0.000000,0.0,0.000

# End

In [204]:
set(inc_exp['OE']).symmetric_difference(set(planned['OE - Plan']))

{'AGF (Div AGCS)',
 'ARAP',
 'ARAP (DE1288)',
 'ART (PharmChem)',
 'ART AG',
 'ART Liechtenstein AG',
 'ART NA Bermuda',
 'ART, AGCS Business',
 'Adjstmt Unit AGCS SE',
 'Argentina',
 'Bermuda Branch',
 'Bulgaria',
 'Chile',
 'Colombia',
 'Croatia',
 'Czech Republic',
 'Denmark',
 'Dummy  Flow Through',
 'Elimination AGCS',
 'Germany-Africa',
 'Hungary',
 'Ireland',
 'Korean Branch',
 'Labuan',
 'MR Brazil',
 'Mexico',
 'Nordic Branch',
 'Poland',
 'Portugal',
 'REST',
 'Reins biz not by OE',
 'Reinsured Bus',
 'Romania',
 'Russia',
 'Slovakia',
 'South Korea',
 'Turkey (Actuals)',
 'Ukraine'}

In [205]:
set(inc_exp['OE'])

{'ART AG',
 'Australia',
 'Austria',
 'Belgium',
 'Brazil',
 'Canada',
 'China',
 'Denmark',
 'Dubai',
 'France',
 'Germany',
 'Germany-Africa',
 'Hong Kong',
 'India',
 'Italy',
 'Japan',
 'Netherlands',
 'New Zealand',
 'Singapore',
 'South Africa',
 'South Korea',
 'Spain',
 'Sweden',
 'Switzerland',
 'UK',
 'US'}

In [206]:
set(planned['OE - Plan'])

{'AGF (Div AGCS)',
 'ARAP',
 'ARAP (DE1288)',
 'ART (PharmChem)',
 'ART Liechtenstein AG',
 'ART NA Bermuda',
 'ART, AGCS Business',
 'Adjstmt Unit AGCS SE',
 'Argentina',
 'Australia',
 'Austria',
 'Belgium',
 'Bermuda Branch',
 'Brazil',
 'Bulgaria',
 'Canada',
 'Chile',
 'China',
 'Colombia',
 'Croatia',
 'Czech Republic',
 'Dubai',
 'Dummy  Flow Through',
 'Elimination AGCS',
 'France',
 'Germany',
 'Hong Kong',
 'Hungary',
 'India',
 'Ireland',
 'Italy',
 'Japan',
 'Korean Branch',
 'Labuan',
 'MR Brazil',
 'Mexico',
 'Netherlands',
 'New Zealand',
 'Nordic Branch',
 'Poland',
 'Portugal',
 'REST',
 'Reins biz not by OE',
 'Reinsured Bus',
 'Romania',
 'Russia',
 'Singapore',
 'Slovakia',
 'South Africa',
 'Spain',
 'Sweden',
 'Switzerland',
 'Turkey (Actuals)',
 'UK',
 'US',
 'Ukraine'}

In [245]:
len(set(inc_exp['Branch Code'].dropna()).symmetric_difference(set(planned['Region'])))
#set(inc_exp['Branch Code'].str[:2].dropna())

23

In [236]:
set(inc_exp['Actuarial Branch'])

{'BR', 'CA', 'CH03', 'CH__', 'DE', 'EXDE', 'SA', 'US'}

In [237]:
set(planned['Region'])

{'AE',
 'AR',
 'AT',
 'AU',
 'BE',
 'BG',
 'BR',
 'CA',
 'CH',
 'CL',
 'CN',
 'CO',
 'CZ',
 'DE',
 'DK',
 'EL',
 'ES',
 'FR',
 'HK',
 'HR',
 'HU',
 'IE',
 'IN',
 'IT',
 'JP',
 'KR',
 'LI',
 'MX',
 'MY',
 'NL',
 'NZ',
 'PL',
 'PT',
 'RE',
 'RO',
 'RU',
 'SE',
 'SG',
 'SK',
 'TR',
 'UA',
 'UK',
 'US',
 'XX',
 'ZA'}

In [141]:
#set(planned['Region'])

In [21]:
# Actuarial vs. Planned
set(inc_exp['OE']).symmetric_difference(set(planned['OE']))

{'AGCS Brazil',
 'AGCS France',
 'AGCS North America',
 'AGCS South Africa',
 'AGF (Div AGCS)',
 'ARAP',
 'ARAP (DE1288)',
 'ART (PharmChem)',
 'ART AG',
 'ART Liechtenstein AG',
 'ART NA Bermuda',
 'ART New York',
 'ART Swiss Branch',
 'ART, AGCS Business',
 'Adjstmt Unit AGCS SE',
 'Argentina',
 'Austria',
 'Austrian Branch',
 'Austrian Run-Off',
 'Belgium',
 'Belgium Branch',
 'Belgium Run-Off',
 'Bermuda Branch',
 'Brazil',
 'Brazil R/I',
 'Bulgaria',
 'CHINA',
 'Chile',
 'China',
 'Colombia',
 'Croatia',
 'Czech Republic',
 'Denmark',
 'Dubai',
 'Dubai Run-Off',
 'Dummy  Flow Through',
 'Elimination AGCS',
 'FFIC',
 'France',
 'French Branch',
 'German Branch',
 'Germany',
 'Germany-Africa',
 'Hong Kong',
 'Hong Kong Branch',
 'Hong Kong Run-Off',
 'Hungary',
 'INDIA',
 'India',
 'Indian Branch',
 'Indian Pre-Branch',
 'Ireland',
 'Italian Branch',
 'Italian Run-Off',
 'Italy',
 'Korean Branch',
 'Labuan',
 'MR Brazil',
 'Mexico',
 'Netherlands',
 'Netherlands Branch',
 'Netherlan

In [143]:
# Actuarial vs. Planned
set(inc_exp['Actuarial Sub Line of Business']).symmetric_difference(set(planned['Sub Line of Business']))

{'Motor General'}

In [144]:
#set(inc_exp['Actuarial Sub Line of Business'])

In [145]:
#set(planned['Sub Line of Business'])   # planned

In [313]:
#set(inc_exp['Sub Line of Business']).intersection(set(final['Sub Line of Business']))

In [154]:
#set(inc_exp['OE'])

In [155]:
#set(planned['OE'])
#set(planned['Region'])
#planned.columns

In [27]:
a = final['LoB'] == 'Financial Lines Prof'
b = final['OE'] == "Spanish Branch"
final[a & b].head(200)

,LoB,OE,Region,Layer,Type of Information,12M,Year,Month,Time,EUR
36,Financial Lines Prof,Spanish Branch,ES01,GROSS NET FULLY CONS,CLAIMS INCURRED C.A.,P 12M 2018,2018,1,2018-01,-126221
37,Financial Lines Prof,Spanish Branch,ES01,GROSS NET FULLY CONS,CLAIMS INCURRED C.A.,P 12M 2018,2018,2,2018-02,-135657
38,Financial Lines Prof,Spanish Branch,ES01,GROSS NET FULLY CONS,CLAIMS INCURRED C.A.,P 12M 2018,2018,3,2018-03,-64935.8
39,Financial Lines Prof,Spanish Branch,ES01,GROSS NET FULLY CONS,CLAIMS INCURRED C.A.,P 12M 2018,2018,4,2018-04,-76393.6
40,Financial Lines Prof,Spanish Branch,ES01,GROSS NET FULLY CONS,CLAIMS INCURRED C.A.,P 12M 2018,2018,5,2018-05,-87850.4
41,Financial Lines Prof,Spanish Branch,ES01,GROSS NET FULLY CONS,CLAIMS INCURRED C.A.,P 12M 2018,2018,6,2018-06,-87537.2
42,Financial Lines Prof,Spanish Branch,ES01,GROSS NET FULLY CONS,CLAIMS INCURRED C.A.,P 12M 2018,2018,7,2018-07,-98804.3
43,Financial Lines Prof,Spanish Branch,ES01,GROSS NET FULLY CONS,CLAIMS INCURRED C.A.,P 12M 2018,2018,8,2018-08,-107417
44,Financial Lines Prof,Spanish Branch,ES01,GROSS NET FULLY CONS,CLAIMS INCURRED C.A.,P 12M 2018,2018,9,2018-09,-116030
45,Financial Lines Prof,Spanish Branch,ES01,GROSS NET FULLY CONS,CLAIMS INCURRED C.A.,P 12M 2018,2018,10,2018-10,-124653


In [62]:
inc_exp.columns 

Index(['Region', 'OE', 'Line of Business', 'Sub Line of Business',
       'Source System', 'Actuarial Region', 'Actuarial Sub Line of Business',
       'Actuarial Branch', 'Branch Code', 'Legal Entity', 'time',
       'Loss Category', 'Accident year', 'target', 'Legal Entity - PAY',
       'Branch - PAY', 'Region - PAY', 'Line of Business - PAY',
       'Sub Line of Business - PAY', 'Classifier - PAY', 'Layer - PAY',
       'Currency - PAY', 'FX rate - PAY',
       'Expected Movement - Attritional - PAY',
       'Expected Movement - Medium - PAY', 'Expected Movement - Large - PAY',
       'Expected Movement - Cat - PAY', 'time - PAY',
       'Expected Movement - Attritional - EUR - PAY',
       'Expected Movement - Large - EUR - PAY',
       'Expected Movement - Medium - EUR - PAY',
       'Expected Movement - Cat - EUR - PAY', 'Legal Entity - CAY',
       'Branch - CAY', 'Region - CAY', 'Line of Business - CAY',
       'Sub Line of Business - CAY', 'Classifier - CAY', 'Layer - CAY',
 

In [24]:
import collections

freq = collections.Counter(full_flat['Region - StrLen'].to_list())
freq

Counter({4: 453600,
         6: 34488,
         5: 32472,
         15: 11736,
         7: 30240,
         11: 23184,
         12: 11592,
         14: 12600,
         17: 4536,
         24: 12600,
         2: 12888,
         13: 37008,
         22: 7128,
         9: 23328,
         8: 12960,
         20: 13176,
         18: 432})

In [25]:
for letter, count in freq.most_common(3):
    print(letter, count)

4 453600
13 37008
6 34488


In [ ]:
df[df.shape[1]-1].tail(20)

### Write to Postgres:

In [ ]:
pg = file.PostgresService(verbose=False)

pg.doWrite(full_flat, "Planned_CAY_PAY")    # if_exists = "append"

In [ ]:
filename = util.get_newest_file(search_for = "AGCS CCO CRA - Monthly Incurred Movements",  src_dir=glob.UC_DATA_DIR)
filename

xls = file.XLSXService(path=filename, root_path=glob.UC_DATA_DIR, dtype= {'time': str}, sheetname='data', index_col=None, header=0)

data_orig = xls.doRead()

data_orig.shape

data_orig.head(10)

In [ ]:
print(data_cay.shape)
print(data_pay.shape)

In [ ]:
joined = data_cay.merge(data_pay, how="left", left_on = ['Legal Entity - CAY', 'Region - CAY', 
                                                         'Line of Business - CAY', 'Sub Line of Business - CAY', 
                                                         'Classifier - CAY', 'Layer - CAY', "time - CAY"], 
                        right_on = ['Legal Entity - PAY', 'Region - PAY',  
                                    'Line of Business - PAY', 'Sub Line of Business - PAY', 
                                    'Classifier - PAY', 'Layer - PAY', "time - PAY"])
print(joined.shape)

In [ ]:
joined.head()

In [ ]:
#ym = file.YAMLservice(root_path = glob.UC_CODE_DIR+"claims_reporting/resources")
#ym.doRead('reference_columns.yaml')
#ym.doWrite(X = {'XLS_columns' : list(data_orig.columns)}, filename = 'reference_columns.yaml')

In [ ]:
i = 0
month = 2

# Make folder name (dependent on year):
fpath = glob.UC_DATA_DIR+"Expected_amnt/"+str(file_input_years[i])

# Make file search name:
str_fname = "AGCS ACT AvsE"+" "+str(file_input_years[i])+"."+"{:02d}".format(month)    
str_fname

In [ ]:
filename = util.get_newest_file(search_for = str_fname,  src_dir=fpath)
filename

In [ ]:
xls = file.XLSXService(path=filename, root_path=fpath, sheetname='Data_CAY', index_col=None, header=2)

data_orig = xls.doRead()

In [ ]:
data_orig.shape

In [ ]:
data_orig.head()

In [ ]:
data_orig.info(verbose=True)

In [ ]:
periodicity = 12

if periodicity == 52:
    config_input = config.in_out52['input']
    config_output = config.in_out52['output']
    config_detect = config.in_out52['detection']
if periodicity == 12:    
    config_input = config.in_out12['input']
    config_output = config.in_out12['output']
    config_detect = config.in_out12['detection']

In [ ]:
filename = util.get_newest_file(search_for = "AGCS CCO CRA - Monthly Incurred Movements",  src_dir=glob.UC_DATA_DIR)
filename

xls = file.XLSXService(path=filename, root_path=glob.UC_DATA_DIR, dtype= {'time': str}, sheetname='data', index_col=None, header=0)

data_orig = xls.doRead()

data_orig.shape

data_orig.head()

In [ ]:
from claims_reporting.resources import preprocessor_incurred as monthly
from claims_reporting.resources import trainer

reload(trainer)
reload(monthly)

outlier_filter = config_detect['training']['outlier_filter']
hyper_para = config_detect['training']['hyper_para']
stat_transform = config_detect['training']['stat_transform']

# Instantiate class:
#--------------------
claims = monthly.claims_reporting(ts_col = 'target')

aggreg_level, pre_filter, ignore_week_lag, min_sample_size, min_median_cnts = list(config_detect['preprocessing'].values())

gen = claims.process_data(data_orig, aggreg_level = 'all_combi', ignore_lag = 1, min_sample_size = 30)

In [ ]:
from adtk.visualization import plot
import statsmodels.api as sm
import matplotlib.ticker as ticker

# Get next series
#-------------------
label, sub_set = next(gen)

print('Claims from period {} to {}.'.format(claims.min_year_period, claims.max_year_period)) 

print(label, sub_set.shape[0])
df = deepcopy(sub_set)

train = trainer.trainer(periodicity = periodicity, verbose=False)
fitted = train.fit(df = df)

y = fitted.ts_values
#y = fitted.val_series
out = fitted.predict(detect_thresh = None)

where = np.where(np.array(claims.time_index) == outlier_filter)[0][0]
outlier_search_list = claims.time_index[where:]

filtered_outliers = []
if out.nof_outliers > 0:
    outlier_dates = out.outlier_dates
    filt = [outl in outlier_search_list for outl in outlier_dates]
    filtered_outliers = np.array(outlier_dates)[filt].tolist()
    
    if len(filtered_outliers) > 0:
        #print("\nSeries",i)
        #print(label, sub_set.shape[0])
        print("Anomaly found!")
        print(filtered_outliers)
    
#lag = 1
#y_diff = util.difference(y, lag)
# First diff.
#util.ts_plot(x=x[lag:], y=y_diff, title='Weekly claim counts (First diff.): '+label) 

# Detect anomalies:
#----------------------
inside = ''    
if label in list(claims.level_wise_aggr.keys()):

    inside = claims.level_wise_aggr[label]       # then shows over which set it was aggregated    
    #new_inside = [str(i)+'\n' for i in inside] 
    
    #main = label +':\n\n '+ str(len(filtered_outliers)) + \
    #    ' outlier(s) detected!\n' + 'Occured at year-calendar week(s): '+ \
    #    ', '.join(filtered_outliers)+'\n'+'Aggregated over:'+str(new_inside)+'\n'
    
    main = label +':\n\n '+ str(len(filtered_outliers)) + \
            ' outlier(s) detected!\n' + 'Occured at year-period(s): '+ \
            ', '.join(filtered_outliers)+'\n'+'\nAggregated over: '
    for i in inside: main += str(i)+'\n'
    
else:
    main = label +':\n\n '+ str(len(filtered_outliers)) + \
        ' outlier(s) detected!\n' + 'Occured at year-period(s): '+ \
        ', '.join(filtered_outliers)+'\n'

    
pp = plot(fitted.val_series, anomaly_true = fitted.anomalies, ts_linewidth=1.2, ts_markersize=6, 
     at_markersize=5, at_color='red', freq_as_period=False, ts_alpha=0.8, at_alpha=0.5, title = main)


# other change point detection algos
#model = "rbf"  # "l2", "rbf"
#signal = train.s_deseasonal.to_numpy()
#signal = y.to_numpy()
#algo = rpt.Pelt(model=model, min_size=5).fit(signal)
#algo = rpt.Binseg(model=model).fit(signal)
#my_bkps = algo.predict(pen=np.log(len(signal))*np.std(signal)**2)

#ind = np.zeros(len(fitted.anomalies))
#ind[np.array(my_bkps)-1] = 1
#anomalies_algo = pd.Series(ind,name=fitted.anomalies.name, index=fitted.anomalies.index)
#bb = plot(fitted.val_series, anomaly_true = anomalies_algo, ts_linewidth=1.2, ts_markersize=6, 
#     at_markersize=5, at_color='red', freq_as_period=False, ts_alpha=0.8, at_alpha=0.5, title = "Algo")

#ticklabels = pp.get_xticks().tolist()
#pp.set_xticklabels(df['time'].tolist())
#pp.get_xticklabels()
#pp.get_xticks().tolist()
#pp.set_xticklabels(a)

# Anomaly probabilities:
#-------------------------
plt.figure(figsize=(12,4), dpi=100)
pro = plt.plot(fitted.anomaly_proba.index, fitted.anomaly_proba, color='tab:blue',label="prob. of anomaly", linestyle='--', marker='o', markerfacecolor='orange', linewidth=1)
plt.plot(fitted.anomaly_proba.index, [fitted.detect_thresh]*len(fitted.anomaly_proba.index), label="decision threshold",  linewidth=.5)
plt.gca().set(title="", xlabel="time", ylabel="probability", ylim = plt.ylim(-0.02, 1.05))   #plt.xlim(left=0)
locs, labels = plt.xticks()
#plt.xticks(y.index[0::3], labels=df['time'].to_numpy()[0::3], rotation=60)
plt.title(r'Anomaly probabilities $\pi_{t}, t=1,...,T$', fontdict = {'fontsize' : 14})
plt.legend(loc='upper left')
plt.tight_layout()
plt.show()  

#start, end = pp.get_xlim()
#pp.xaxis.set_ticks(np.linspace(start, end, num=len(fitted.df['time'].values.tolist()[0::3])))
#pp.set_xticklabels(fitted.df['time'].values.tolist()[0::3], rotation=65)

#pp.xaxis.set_major_formatter(ticker.FormatStrFormatter('%0.1f'))

# Density plots
#----------------------------------------------------
plt.figure(figsize=(12,3), dpi=100)
#plt.subplot(211)
y.hist(bins=25)
plt.gca().set(title=label, xlabel="target", ylabel="abs. frequency")
#--------------------------------------------------------------------
#plt.subplot(212)
#log_y = np.log(1 + y)
#log_y.hist(bins=15)
#plt.gca().set(title='', xlabel="log counts", ylabel="abs. frequency")
#plt.subplot(212)
#y.plot(kind='kde')
#plt.gca().set(title='', xlabel="counts", ylabel="density")
#------------------------------------------------------------------------
# Draw Boxplot
if periodicity == 52 :
    fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(15,6), dpi= 80)
if periodicity == 12 :    
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15,6), dpi= 80)
sns.boxplot(x='year', y='target', data=sub_set, ax=axes[0])
sns.boxplot(x='month', y='target', data=sub_set, ax=axes[1]).set(ylabel="counts")

if periodicity == 52 :
    sns.boxplot(x='period', y='target', data=sub_set, ax=axes[2], orient='v').set(
    xlabel='week', ylabel="counts")
#------------------------------------------------------------------------------------------

# Set Titles
axes[0].set_title('Yearly box plots\n(Trend)', fontsize=18) 
axes[1].set_title('Monthly box plots\n(Seasonality)', fontsize=18)
if periodicity == 52 :
    axes[2].set_title('Weekly box plots\n(Seasonality)', fontsize=18)
#plt.yticks(rotation=15)
plt.xticks(rotation=45)

plt.show()

# Bayesian:
#trim = 3
#probm = exact_post_cp(y.values[trim:], alpha = .2, beta = .2, gamma = .2, delta = .2)

#x = np.arange(0, len(y))
#fig, ax = plt.subplots()
#rects1 = ax.bar(x, np.append(np.zeros(3), probm), label='posterior', width = 4)
#plt.title('Posterior p.m.f. of change point')
#plt.show()
#plt.plot(x,y,'b')
#plt.axvline(x[np.argmax(probm)], color='k', linestyle='--', lw=.65)

In [ ]:
# Get next series
#-------------------
label, sub_set = next(gen)

print(label, sub_set.shape[0])
df = deepcopy(sub_set)

train = trainer.trainer(verbose=False)
fitted = train.fit(df = df)

y = fitted.ts_values
out = fitted.predict(detect_thresh = None)

In [ ]:
from adtk.detector import OutlierDetector
from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import IsolationForest

outlier_detector = OutlierDetector(LocalOutlierFactor(contamination=0.01))
#outlier_detector = OutlierDetector(IsolationForest(n_estimators = 10))

anomalies = outlier_detector.fit_detect(y.to_frame())
anomalies

In [ ]:
?OutlierDetector

BOCD:

In [ ]:
from claims_reporting.resources import bocd

reload(bocd)

T = len(y)
hazard = .01  # Constant prior on changepoint probability.
mean0  = 0      # The prior mean on the mean parameter.
var0   = 2      # The prior variance for mean parameter.
varx   = 1      # The known variance of the data.
cps = None

model = bocd.GaussianUnknownMean(mean0, var0, varx)
bc = bocd.BayesOCPD(model, hazard, mini_run_length = 2)

In [ ]:
from adtk.detector import AutoregressionAD

?AutoregressionAD

In [ ]:
cp_hat = bc.fit_predict(X = y.to_numpy())

R, pmean, pvar, cps_MAP, cps_est = bc.R, bc.pmean, bc.pvar, bc.cps_MAP, bc.cps_est

In [ ]:
bocd.plot_posterior(T, y.to_numpy(), cps, cps_est, R, pmean, pvar, True)

# Run all

In [ ]:
from claims_reporting.utils import tsa_utils as tsa
from claims_reporting.utils import utils_func as util
from claims_reporting.services import file

from claims_reporting.config import global_config as glob
from claims_reporting.services import file
from claims_reporting.resources import config

from adtk.visualization import plot
import statsmodels.api as sm

reload(tsa)
reload(file)
reload(config)
reload(glob)
reload(util)

In [ ]:
train = trainer.trainer(verbose=False)

results, results_new = train.run_all(data_orig = data_orig, aggreg_level = 'all_combi', write_table = False, verbose=True, ignore_lag = 1, min_sample_size = 20)

#results = deepcopy(results_new)

In [ ]:
new_anomalies = list(set(results_new['time_series_name']))

In [ ]:
train.print_anomalies(search_term = "Marine")

In [ ]:
#print('Number of anomalies in database: {}'.format(train.anomaly_history.shape[0]))

In [ ]:
for label, val in train.filt_suspects_plot.items():
    
    if label in new_anomalies:

        fitted_val_series = val['val_series']
        y = fitted_val_series
        fitted_anomalies = val['anom_flag']
        anomaly_proba = val['anom_evidence']
        filtered_outliers = train.filt_suspects_values[label]['anomaly_dates']
        sub_set = train.filt_suspects_plot[label]['df']

        inside = ''    
        if label in train.level_wise_aggr:    
            inside = train.level_wise_aggr[label]       # then shows over which set it was aggregated    
            main = label +':\n\n '+ str(len(filtered_outliers)) + \
                    ' outlier(s) detected!\n' + 'Occured at year-calendar week(s): '+ \
                    ', '.join(filtered_outliers)+'\n'+'\nAggregated over: '
            for i in inside: main += str(i)+'\n'
        else:
            main = 'Plot 1: '+label +':\n\n '+ str(len(filtered_outliers)) + \
                ' outlier(s) detected!\n' + 'Occured at year-calendar week(s): '+ \
                ', '.join(filtered_outliers)+'\n'
            
        # Plot 1:
        #---------
        p1 = plot(fitted_val_series, anomaly_true = fitted_anomalies, ts_linewidth=1.2, ts_markersize=6, 
             at_markersize=5, at_color='red', freq_as_period=False, ts_alpha=0.8, at_alpha=0.5, 
             title = main)

        plt.figure(figsize=(1,1))
        plt.text(0.05, 0.05, '', ha='center', va='center', size=.1, wrap=True)
        plt.axis('off')
        plt.show() 
        
        # Anomaly probabilities:
        #-------------------------
        plt.figure(figsize=(12,4), dpi=100)
        pro = plt.plot(anomaly_proba.index, anomaly_proba, color='tab:blue',label="prob. of anomaly", linestyle='--', marker='o', markerfacecolor='orange', linewidth=1)
        plt.plot(anomaly_proba.index, [train.detect_thresh]*len(anomaly_proba.index), label="decision threshold",  linewidth=.5)
        plt.gca().set(title="", xlabel="time", ylabel="probability", ylim = plt.ylim(-0.02, 1.05))   #plt.xlim(left=0)
        locs, labels = plt.xticks()
        #plt.xticks(y.index[0::3], labels=df['time'].to_numpy()[0::3], rotation=60)
        plt.title(r'Anomaly probabilities $\pi_{t}, t=1,...,T$', fontdict = {'fontsize' : 14})
        plt.legend(loc='upper left')
        plt.tight_layout()
        plt.show()          
        
        # Density plots
        #----------------------------------------------------
        plt.figure(figsize=(12,3), dpi=100)
        #plt.subplot(211)
        y.hist(bins=20)
        plt.gca().set(title=label, xlabel="target", ylabel="abs. frequency")
        #--------------------------------------------------------------------
        
        # Plot 2 + 3:
        #--------------------------------------------------------------------------
        # Draw Boxplot
        fig, axes = plt.subplots(1, 2, figsize=(14.5,5.5), dpi= 80)
        sns.boxplot(x='year', y='target', data=sub_set, ax=axes[0])
        sns.boxplot(x='period', y='target', data=sub_set)

        axes[0].set_title('Plot 2: Year-wise Box plots (Trend)', fontsize=14.5) 
        axes[1].set_title('Plot 3: Week-wise Box plots (Seasonality)', fontsize=14.5)
        plt.xticks(rotation=65)
        fig.tight_layout()
        plt.show()

        """
        
        # Forecast time series:
        #-------------------------------------------------------------    
        #n_step_ahead = list(config_detect['prediction'].values())[0]
        #endog = np.log(1 + y).to_numpy()
        endog = y
        mod_uc = sm.tsa.UnobservedComponents(endog, 
                                             'smooth trend',
                                             #'local linear trend', 
                                             cycle=True, 
                                             seasonal=52, 
                                             irregular=False,
                                             #stochastic_level=True, 
                                             stochastic_cycle=True, 
                                             damped_cycle=False)

        res_uc = mod_uc.fit(method='powell', disp=False)
        #res_uc = mod_uc.fit(disp=False, cov_type = "robust_approx")

        # Plot 4:
        #----------
        fig = res_uc.plot_components(level=False, trend=False, seasonal=False, cycle=False, 
                                     legend_loc='upper left', figsize=(15, 6))
        axes = fig.get_axes()
        axes[0].set_xlabel('time index')
        axes[0].set_title("Plot 4: Forecast number of claims - "+label, fontsize=14.5)
        a = axes[0].set_ylabel('claim counts')

        # Logging
        plt.figure(figsize=(1,1))
        plt.text(0.05, 0.05, '', ha='center', va='center', size=.1, wrap=True)
        plt.axis('off')
        plt.show() 
        
        """